# Assignment 1: Text Preprocessing

Nikolaou Nikolaos - DSC18014

# Question A

In [19]:
import nltk
import string
import requests
from readability.readability import Document
from bs4 import BeautifulSoup
import codecs
import os
# nltk.download('averaged_perceptron_tagger')
# nltk.download('stopwords')
# nltk.download('punkt')
import nltk.data
# nltk.data.path.append("/mnt/c/users/nick/Google Drive/Study/DataScience/2. NLP/Assignments/Exercise 1/.nltk_data/")
# nltk.data.path.append("C:/Users/nnikolao/PycharmProjects/.nltk_data/")

[nltk_data] Downloading package stopwords to /home/nick/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


What is the word count and vocabulary of this Web page?

In [5]:
url = 'https://en.wikipedia.org/wiki/Artificial_neural_network'

In [6]:
r = requests.get(url)
html = r.text

In [7]:
TAGS = ['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'h7', 'p', 'li']

In [47]:
def preprocess(f):
    paper = Document(f)
    soup = BeautifulSoup(paper.summary())
    # kill all script and style elements
    for script in soup(["script", "style","math"]):
        script.decompose()    # rip it out
    output = [paper.title()]
    for tag in soup.find_all(TAGS):
        # Get the HTML node text
        paragraph = tag.get_text()

        # Sentence Tokenize
        sentences = nltk.sent_tokenize(paragraph)
        for idx, sentence in enumerate(sentences):
            # Word Tokenize and Part of Speech Tagging
            sentences[idx] = nltk.pos_tag(nltk.word_tokenize(sentence))

        # Yield a list of sentences (the paragraph); each sentence of
        # which is a list of tuples in the form (token, tag).
        yield sentences

In [48]:
try:
    os.mkdir("./nlp_data")
except OSError:  
    print ("Directory already exists")
outpath = os.path.join("./nlp_data/", "outfile" + ".txt")
with codecs.open(outpath, 'w+', encoding='utf-8') as f:
    # Write paragraphs double newline separated and sentences
    # separated by a single newline. Also write token/tag pairs.
    for paragraph in preprocess(html):
        for sentence in paragraph:
            f.write(" ".join("%s/%s" % (word, tag) for word, tag in sentence))
            f.write("\n")
            f.write("\n")

Directory already exists


Now in the Outfile, we have the preprocessed document

In [49]:
# Construct stopwords
stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(list(string.punctuation))          # Remove punctuation
stopwords.extend(["''", '``', "'s", "n't", "'ll"])  # Custom stopwords

In [50]:
# Construct the corpus
corpus = nltk.corpus.TaggedCorpusReader("./nlp_data",r'.*txt')

In [51]:
# Get the interesting words from corpus
words = [word.lower() for word in corpus.words() if word not in stopwords]

In [52]:
# Count the words,sentences and tags
tokens    = nltk.FreqDist(corpus.words())
unigrams  = nltk.FreqDist(words)
bigrams   = nltk.FreqDist(nltk.bigrams(words))
tags      = nltk.FreqDist(tag for word, tag in corpus.tagged_words())

In [53]:
# Eliminate stopwords
for word in stopwords:
    unigrams.pop(word, None)
    bigrams.pop(word, None)

In [54]:
# Enumerate the vocabulary and word count
vocab     = len(tokens)            # The number of unique tokens
sents     = len(corpus.sents())
count     = sum(tokens.values())   # The word count for the entire corpus

Answering the questions:

In [55]:
print("1. What is the word count and vocabulary of this Web page?")
print (" This corpus contains %i words with a vocabulary of %i tokens."  % (count, vocab))

print("\n2. How many sentences are contained in the page?")
print (" This corpus contains %i sentences."  % (sents))

print("\n3. What is the lexical diversity of the page?")
print (" The lexical diversity is %0.3f" % (float(count) / float(vocab)))

1. What is the word count and vocabulary of this Web page?
 This corpus contains 23153 words with a vocabulary of 4785 tokens.

2. How many sentences are contained in the page?
 This corpus contains 1971 sentences.

3. What is the lexical diversity of the page?
 The lexical diversity is 4.839


In [56]:
print("4. What are the 5 most common lexical categories (parts of speech)?")
print (" The 5 most common tags are:")
for idx, tag in enumerate(tags.most_common(5)):
    print ("    %i. %s (%i samples)" % ((idx+1,) + tag))

4. What are the 5 most common lexical categories (parts of speech)?
 The 5 most common tags are:
    1. NNP (3517 samples)
    2. NN (3284 samples)
    3. JJ (1859 samples)
    4. . (1792 samples)
    5. IN (1672 samples)


In [57]:
print("5. What are the 10 most common unigrams, the 10 most common bigrams?")
print (" The 10 most common unigrams are:")
for idx, tag in enumerate(unigrams.most_common(10)):
    print ("    %i. %s (%i samples)" % ((idx+1,) + tag))
print ("\n The 10 most common bigrams are:")
for idx, tag in enumerate(bigrams.most_common(10)):
    print ("    %i. %s (%i samples)" % ((idx+1,) + tag))

5. What are the 10 most common unigrams, the 10 most common bigrams?
 The 10 most common unigrams are:
    1. neural (271 samples)
    2. networks (213 samples)
    3. learning (191 samples)
    4. network (129 samples)
    5. deep (90 samples)
    6. artificial (80 samples)
    7. function (72 samples)
    8. edit (70 samples)
    9. recognition (55 samples)
    10. using (55 samples)

 The 10 most common bigrams are:
    1. ('neural', 'networks') (147 samples)
    2. ('neural', 'network') (68 samples)
    3. ('artificial', 'neural') (58 samples)
    4. ('machine', 'learning') (23 samples)
    5. ('deep', 'learning') (23 samples)
    6. ('international', 'conference') (17 samples)
    7. ('cost', 'function') (16 samples)
    8. ('pattern', 'recognition') (14 samples)
    9. ('ieee', 'transactions') (13 samples)
    10. ('recurrent', 'neural') (12 samples)


In [58]:
print("6. How many nouns are in the page?")
print (" There are %i nouns in the corpus" % sum(val for key,val in tags.items() if key.startswith('N')))

6. How many nouns are in the page?
 There are 8044 nouns in the corpus


# Question B

In [32]:
import csv

In [33]:
csv_outpath = os.path.join("./nlp_data/", "csv_outfile" + ".csv")
with codecs.open(csv_outpath, 'w', encoding='utf-8-sig') as csv_file:
    csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL, dialect='excel')
    csv_writer.writerow(['Νικόλαος', 'Νικολάου','“Μεταπτυχιακό στην Επιστήμη Δεδομένων”'])